## Library Imports

In [1]:

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing the list of stocks

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquire API token

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making API call

In [5]:
symbol = "AAPL"
url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(url).json()
print(data)

{'avgTotalVolume': 109294790, 'calculationPrice': 'close', 'change': 1.25, 'changePercent': 0.00711, 'close': 0, 'closeSource': 'lafcoiif', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': ' 1  ryaidepuenetme5lidc', 'highTime': 1719558164027, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 179.14, 'iexCloseTime': 1691007725524, 'iexLastUpdated': 1725332045507, 'iexMarketPercent': 0.01579085508282332, 'iexOpen': 176.66, 'iexOpenTime': 1653660892804, 'iexRealtimePrice': 178.5, 'iexRealtimeSize': 25, 'iexVolume': 1351103, 'lastTradeTime': 1720884270676, 'latestPrice': 178.77, 'latestSource': 'Close', 'latestTime': 'February 2, 2022', 'latestUpdate': 1670552675248, 'latestVolume': None, 'low': 0, 'lowSource': ' eedindulcp ar 5y1eteim', 'lowTime': 1682382993777, 'ma

## Parsing API call

In [6]:
price = data["latestPrice"]
market_cap = data["marketCap"]

## Adding stock to DataFrame

In [7]:
columns = ["Ticker","Stock Price","Market Capitalization","Numbers Of Shares to buy"]
dataFrame = pd.DataFrame(columns=columns)

In [8]:
dataFrame.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            "N/A"
        ],
        index = columns
    
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Numbers Of Shares to buy
0,AAPL,178.77,2928088119354,N/A


## Loop through the tickers in list of stocks

In [ ]:
dataFrame = pd.DataFrame(columns=columns)
for stock in stocks["Ticker"]:
    url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(url).json()
    dataFrame = dataFrame.append(
        pd.Series(
            [
                stock,
                data["latestPrice"],
                data["marketCap"],
                "N/A"
            ],
            index=columns
        ),
        ignore_index=True
    )

In [11]:
dataFrame

,Ticker,Stock Price,Market Capitalization,Numbers Of Shares to buy
0,A,149.73,45413995036,N/A
1,AAL,17.48,11371711002,N/A
2,AAP,243.47,14985592815,N/A
3,AAPL,178.10,2925064876577,N/A
4,ABBV,141.77,255343243636,N/A


## Batch API calls 

In [13]:
def chunks(lst, n):
    #Yield succesive n-sized chunks from list
    for i in range(0,len(lst),n):
        yield lst[i:i +n]

In [60]:
symbol_groups = list(chunks(stocks["Ticker"],100))
symbol_strings = []

for i in range(0,len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
dataFrame = pd.DataFrame(columns = columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        if symbol == "WLTW":
            continue
        dataFrame = dataFrame.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']["marketCap"],
                "N/A"
            ],
            index=columns
        ),
        ignore_index=True
    )
dataFrame

,Ticker,Stock Price,Market Capitalization,Numbers Of Shares to buy
0,A,144.53,43631467141,N/A
1,AAL,17.15,11327238146,N/A
2,AAP,240.09,15488005397,N/A
3,AAPL,180.65,2924477718180,N/A
4,ABBV,139.43,251535804015,N/A
...,...,...,...,...
499,YUM,129.33,36882605468,N/A
500,ZBH,126.17,25729375626,N/A
501,ZBRA,536.50,27876680818,N/A
502,ZION,69.70,10973728607,N/A


## Calculating the number of shares to buy

In [62]:
portfolio_size = input("Enter the value of your porfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("ERROR!!! Please type integer")
    portfolio_size= input("Please type integer")
    val = portfolio_size

Enter the value of your porfolio: 10000000


In [63]:
position_size = val /len(dataFrame.index)
for i in range(0,len(dataFrame.index)):
    dataFrame.loc[i,"Numbers Of Shares to buy"] = math.floor(position_size / dataFrame.loc[i,"Stock Price"])
dataFrame

,Ticker,Stock Price,Market Capitalization,Numbers Of Shares to buy
0,A,144.53,43631467141,137
1,AAL,17.15,11327238146,1156
2,AAP,240.09,15488005397,82
3,AAPL,180.65,2924477718180,109
4,ABBV,139.43,251535804015,142
...,...,...,...,...
499,YUM,129.33,36882605468,153
500,ZBH,126.17,25729375626,157
501,ZBRA,536.50,27876680818,36
502,ZION,69.70,10973728607,284


## Formatting Excel Ouput

### Initiliazing XlsxWriter Object

In [83]:
writer = pd.ExcelWriter("Recommended trades.xlsx",engine="xlsxwriter")
dataFrame.to_excel(writer, "Recommended Trades",index = False)

### Creating needed formats for .xlsx file

In [84]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

dollar_format = writer.book.add_format(
    
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    
    {
        'num_format': '0',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

### Applying Formats to the Columns in .xlsx File

In [85]:
column_formats = {
    "A":["Ticker",string_format],
    "B":["Stock Price",dollar_format],
    "C":["Market Capitialization",dollar_format],
    "D":["Numbers Of Shares to buy",integer_format]
    
}
for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f'{column}:{column}',30,column_formats[column][1])
    writer.sheets["Recommended Trades"].write(f"{column}1",column_formats[column][0],column_formats[column][1])  

## Save Excel file

In [ ]:
writer.save()